<a href="https://colab.research.google.com/github/ReidelVichot/LC_identification/blob/main/EmpByInd_112624.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. PROBLEM DEFINITION

**Background**

“A logistics cluster (LC) is defined as the geographical concentration of firms providing logistics services, such as transportation carriers, warehousing providers, third-party logistics (3PL-s), and forwarders, as well as some other enterprises that are mainly in the logistics business, including logistics enterprises to provide services to various industries” (Rivera et al., 2014, p. 223).  

Several relevant scholars in the field of logistics claim that clustering logistic activity has a positive impact on the efficiency of the economic activity, reduction of costs, and increase of collaboration among the firms that belong to the cluster (Rivera et al., 2014; Rivera, Gligor, et al., 2016; Rivera, Sheffi, et al., 2016; Sheffi, 2013, 2012). Although some of these authors mention that some of these benefits require some trade-offs (Rivera, Gligor, et al., 2016), these trade-offs are not further explored, resulting in an incomplete understanding of the socio-economic effects of the agglomeration of logistics activity. This becomes more problematic given that governments around the world seem to be embracing the idea of logistics clusters being some sort of panacea for economic development based on supply chain management improvements (Baranowski et al., 2015; Baydar et al., 2019; Chung, 2016), even though empirical studies that assess the role of government spending on the formation of logistics clusters are lacking (Liu et al., 2022). In other words, the field still lacks methodological and theoretical development, resulting in an incomplete understanding of the mechanisms of logistical clustering and their socio-economic effects.

**Problem**

There is not a current database of logistics clusters in the US. However, Rivera et al (2014) designed a method to test logistical agglomeration in US counties using NAICS codes and [CBP](https://www.census.gov/programs-surveys/cbp.html) information. Before conducting analyis on the effects of Logistics Clusters on society and the role of governments in their formation it is necessary to have an accurate picture of all logistics clusters in the US. For this purpose, I will extend Reviera's et al (2014) methodology to all the CBP years in which NAICS codes are used and use this database for future analyses.

#2. DATA COLLECTION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import time

# -- this line is to make pandas future-proof, Copy-on-Write will become the default in pandas 3.0.
pd.options.mode.copy_on_write = True

pd.set_option("future.no_silent_downcasting", True)

# -- Set the data path
dpath = "/content/drive/MyDrive/Disertation/"

In [19]:
t0 = time.time()
for year in range(1998, 2022):
  xx = str(year)[2:]
  fname = dpath + "CBP_data/cbp" + xx + "co/cbp" + xx + "co.txt"
  temp = pd.read_csv(fname)
  if year == 2015:
    temp.columns = temp.columns.str.lower()
  # -- add a year column
  temp["year"] = year
  # -- add a GEOID
  temp["GEOID"] = temp.fipstate.astype(str).str.zfill(2) + temp.fipscty.astype(str).str.zfill(3)
  # -- create a global variable and save a dataframe into it
  temp = temp[temp["naics"].str.endswith("----")]
  temp = temp[~temp["naics"].str.startswith("--")]
  globals()["ind" + xx] = temp
  # -- delete to save RAM
  del temp
t1 = time.time()
print("Execution Time: ", (t1 - t0)/60, " mins")


Execution Time:  2.7614225188891095  mins


#3. DATA PREPARATION

In [20]:
# The CBP record layouts are different accross years.
cols_98_16 = ['fipstate', 'fipscty', 'naics', 'empflag', 'emp', 'ap', 'est',
              'n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
              'n500_999','n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4',
              'year', 'GEOID']

cols_17    = ['fipstate', 'fipscty', 'naics', 'empflag', 'emp', 'ap', 'est',
              'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
              'n500_999','n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4',
              'year', 'GEOID']

cols_18_21 = ['fipstate', 'fipscty', 'naics', 'emp_nf',  'emp', 'ap', 'est',
              'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
              'n500_999','n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4',
              'year', 'GEOID']

t0 = time.time()
for year in range(1998, 2017):
  xx = str(year)[2:]
  globals()["ind" + xx] = globals()["ind" + xx][cols_98_16]

ind17 = ind17[cols_17]
ind17.columns = ind98.columns

#####  NOTE: CBP17 has object-type columns for the employment by establishment size.
# from 2017-2021 there are values that have N instead of 0 for emp by est size
ind17[ind17.columns[7:19]] = ind17[ind17.columns[7:19]].replace("N", 0).astype(int)

for year in range(2018, 2022):
  xx = str(year)[2:]
  globals()["ind" + xx] = globals()["ind" + xx][cols_18_21]
  globals()["ind" + xx][globals()["ind" + xx].columns[7:19]] = globals()["ind" + xx][globals()["ind" + xx].columns[7:19]].replace("N", 0).astype(int)
  globals()["ind" + xx].columns = ind98.columns

# -- Between 1998 and 2017, there was a suppression flag (empflag) that affects
#    the employment. Instead of using employment as it is, I will adjust it
#    based on the middle point of the employment by establishment size.

# -- create an unified dataframe for all years (1998-2017)
frames = [ind98, ind99, ind00, ind01, ind02, ind03, ind04, ind05, ind06, ind07,
          ind08, ind09, ind10, ind11, ind12, ind13, ind14, ind15, ind16, ind17,
          ind18, ind19, ind20, ind21]

ind = pd.concat(frames).reset_index().drop(columns="index")
# -- lambda funditon to estimate employment if the employment is flagged
t0 = time.time()
ind['emp_adj'] = ind.apply(lambda row: row["emp"]
                           if pd.isna(row['empflag'])
                           else row['n1_4']*2.5 + row['n5_9']*7 +
                                row['n10_19']*14.5 + row['n20_49']*34.5 +
                                row['n50_99']*74.5 + row['n100_249']*174.5 +
                                row['n250_499']*374.5 + row['n500_999']*749.5 +
                                row['n1000']*1000 , axis=1)
ind = ind.drop(columns=['n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99',
                        'n100_249', 'n250_499', 'n500_999', 'n1000',
                        'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4'])
ind["emp_adj"] = ind.apply(lambda row: row["emp"] if (row["year"]==2018)|(row["year"]==2019)|(row["year"]==2020)|(row["year"]==2021) else row["emp_adj"], axis=1)

# this cbp is from 1998-2021
ind = ind.groupby(["GEOID", "year", "fipstate", "fipscty", "naics"]).sum(numeric_only=True).reset_index()

# -- save the cvs to use it later
ind.to_csv(dpath + "industries.csv", index = False)

t1 = time.time()
print("Execution Time: ", (t1-t0)/60, " mins")

Execution Time:  1.00745689868927  mins


In [21]:
# Run this line to get access to the most current version of the CBP
ind_temp = pd.read_csv(dpath + "industries.csv")
ind_temp["GEOID"] = ind_temp["GEOID"].astype(str).apply(lambda x: x.zfill(5))
ind = ind_temp
del ind_temp

In [22]:
ind

,GEOID,year,fipstate,fipscty,naics,emp,ap,est,emp_adj
0,01001,1998,1,1,11----,0,0,12,46.5
1,01001,1998,1,1,21----,0,0,2,9.5
2,01001,1998,1,1,22----,101,3454,9,101.0
3,01001,1998,1,1,23----,637,11355,112,637.0
4,01001,1998,1,1,31----,1803,68867,30,1803.0
...,...,...,...,...,...,...,...,...,...
1376894,56999,2021,56,999,54----,215,20989,47,215.0
1376895,56999,2021,56,999,56----,644,20540,29,644.0
1376896,56999,2021,56,999,61----,31,1325,9,31.0
1376897,56999,2021,56,999,62----,17,555,4,17.0


In [24]:
# --Removing non-contiguous US counties
# Alaska, Hawaii, American Samoa, Guam, Nothern Marianas, Puerto Rico,
# Virgin Islands
nc_states = [2, 15, 60, 66, 69, 72, 78]
for i in nc_states:
  print("IND      ", len(ind))
  ind = ind[ind["fipstate"] != i]


IND       1376899
IND       1365570
IND       1363486
IND       1363486
IND       1363486
IND       1363486
IND       1363486


#4. MACHINE LEARNING

#5. PROBLEM SOLUTION